# Amazon SageMaker Object Detection for Bird Species


## Setup

Before preparing the data, there are some initial steps required for setup.


This notebook requires two additional Python packages:
* **OpenCV** is required for gathering image sizes and flipping of images horizontally.
* The **MXNet** runtime is required for using the im2rec tool.

This solution relies on a config file to run the provisioned AWS resources. Run the cell below to generate that file.

In [ ]:
import json
import os
import boto3

In [ ]:
client = boto3.client("servicecatalog")
cwd = os.getcwd().split("/")
i = cwd.index("S3Downloads")
pp_name = cwd[i + 1]
pp = client.describe_provisioned_product(Name=pp_name)
record_id = pp["ProvisionedProductDetail"]["LastSuccessfulProvisioningRecordId"]
record = client.describe_record(Id=record_id)

keys = [x["OutputKey"] for x in record["RecordOutputs"] if "OutputKey" and "OutputValue" in x]
values = [x["OutputValue"] for x in record["RecordOutputs"] if "OutputKey" and "OutputValue" in x]
stack_output = dict(zip(keys, values))

with open(f"/root/S3Downloads/{pp_name}/stack_outputs.json", "w") as f:
    json.dump(stack_output, f)

In [ ]:
import sagemaker

sess = sagemaker.Session()

role = sagemaker.get_execution_role()
sagemaker_config = json.load(open("stack_outputs.json"))
demo_endpoint_name = sagemaker_config["DemoEndpointName"]
solution_bucket = sagemaker_config["SolutionS3Bucket"]
region = sagemaker_config["AWSRegion"]
library_version = sagemaker_config["LibraryVersion"]
solution_name = sagemaker_config["SolutionName"]

This is the demo notebook. We've already trained the object detection model and deploy it to a sagemaker endpoint for you. If you want to check the dataset and how we train the model, please proceed to the `object_detection_birds.ipynb` for more details.

## Test the model

Now that the trained model is deployed at an endpoint that is up-and-running, we can use this endpoint for inference.  The results of a call to the inference endpoint are in a format that is similar to the .lst format, with the addition of a confidence score for each detected object. The format of the output can be represented as `[class_index, confidence_score, xmin, ymin, xmax, ymax]`. Typically, we don't visualize low-confidence predictions.

We have provided a script to easily visualize the detection outputs. You can visulize the high-confidence preditions with bounding box by filtering out low-confidence detections using the script below:

In [ ]:
def visualize_detection(img_file, dets, classes=[], thresh=0.6):
    """
    visualize detections in one image
    Parameters:
    ----------
    img : numpy.array
        image, in bgr format
    dets : numpy.array
        ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
        each row is one object
    classes : tuple or list of str
        class names
    thresh : float
        score threshold
    """
    import random
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg

    img = mpimg.imread(img_file)
    plt.imshow(img)
    height = img.shape[0]
    width = img.shape[1]
    colors = dict()
    num_detections = 0
    for det in dets:
        (klass, score, x0, y0, x1, y1) = det
        if score < thresh:
            continue
        num_detections += 1
        cls_id = int(klass)
        if cls_id not in colors:
            colors[cls_id] = (random.random(), random.random(), random.random())
        xmin = int(x0 * width)
        ymin = int(y0 * height)
        xmax = int(x1 * width)
        ymax = int(y1 * height)
        rect = plt.Rectangle(
            (xmin, ymin),
            xmax - xmin,
            ymax - ymin,
            fill=False,
            edgecolor=colors[cls_id],
            linewidth=3.5,
        )
        plt.gca().add_patch(rect)
        class_name = str(cls_id)
        if classes and len(classes) > cls_id:
            class_name = classes[cls_id]
        print("{},{}".format(class_name, score))
        plt.gca().text(
            xmin,
            ymin - 2,
            "{:s} {:.3f}".format(class_name, score),
            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
            fontsize=12,
            color="white",
        )

    print("Number of detections: " + str(num_detections))
    plt.show()

Now we use our endpoint to try to detect objects within an image. Since the image is a jpeg, we use the appropriate content_type to run the prediction. The endpoint returns a JSON object that we can simply load and peek into. We have packaged the prediction code into a function to make it easier to test other images.  Note that we are defaulting the confidence threshold to 30% in our example, as a couple of the birds in our sample images were not being detected as clearly.  Defining an appropriate threshold is entirely dependent on your use case.

In [ ]:
import boto3

runtime = boto3.client(service_name="runtime.sagemaker")


OBJECT_CATEGORIES = ['017.Cardinal',
                     '036.Northern_Flicker',
                     '047.American_Goldfinch',
                     '068.Ruby_throated_Hummingbird',
                     '073.Blue_Jay']


def show_bird_prediction(filename, ep, thresh=0.40):
    b = ""
    with open(filename, "rb") as image:
        f = image.read()
        b = bytearray(f)
    endpoint_response = runtime.invoke_endpoint(EndpointName=ep, ContentType="image/jpeg", Body=b)
    results = endpoint_response["Body"].read()
    detections = json.loads(results)
    visualize_detection(filename, detections["prediction"], OBJECT_CATEGORIES, thresh)

Here we download images that the algorithm has not yet seen.

In [ ]:
from sagemaker.s3 import S3Downloader

test_images_bucket = f"s3://{solution_bucket}-{region}/{library_version}/{solution_name}"
test_images_prefix = f"artifacts/test_images"
test_data = f"{test_images_bucket}/{test_images_prefix}"
print("test data: ")
S3Downloader.list(test_data)

In [ ]:
!aws s3 cp --recursive --no-progress $test_data test_images

We have deployed the demo endpoint for you. You could directly use it and check the result.

In [ ]:
def test_model():
    show_bird_prediction("test_images/hummingbird-1.jpg", demo_endpoint_name)
    show_bird_prediction("test_images/blue-jay-1.jpg", demo_endpoint_name)
    show_bird_prediction("test_images/multi-goldfinch-1.jpg", demo_endpoint_name)
    show_bird_prediction("test_images/northern-flicker-1.jpg", demo_endpoint_name)
    show_bird_prediction("test_images/northern-cardinal-1.jpg", demo_endpoint_name)

test_model()

## Clean up
Here we delete the SageMaker endpoint, as we will no longer be performing any inferences.  This is an important step, as your account is billed for the amount of time an endpoint is running, even when it is idle.

Demo endpoints will be cleaned up after you delete the solution. You could also un-comment the below command to delete the endpoint.

In [1]:
# sagemaker.Session().delete_endpoint(demo_endpoint_name)